In [6]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary

df = pd.read_csv("flujo_efectivo_tienda_dia.csv")
# Calcular PO
df["PO"] = (df["ROP_efectivo"] + df["Max"]) / 2

# Crear modelo
model = LpProblem("Maximizar_Tiendas_Zona_2_3", LpMaximize)

# Variables binarias: ¿Está en zona 2 o 3?
z2 = {i: LpVariable(f"z2_{i}", cat=LpBinary) for i in df.index}
z3 = {i: LpVariable(f"z3_{i}", cat=LpBinary) for i in df.index}

M = 1e6  # Constante grande

# ➕ Restricciones de pertenencia a zona 2 y zona 3
for i in df.index:
    f = df.loc[i, "flujo_neto"]
    ROP = df.loc[i, "ROP_efectivo"]
    PO = df.loc[i, "PO"]
    Max = df.loc[i, "Max"]

    model += f - ROP + M * (1 - z2[i]) >= 0
    model += PO - f + M * (1 - z2[i]) >= 0

    model += f - PO + M * (1 - z3[i]) >= 0
    model += Max - f + M * (1 - z3[i]) >= 0

# 🎯 Objetivo: Max. tiendas en zona 2 o 3
model += lpSum(z2[i] + z3[i] for i in df.index)

# 🧠 Resolver
model.solve()

# 📊 Resultados
df["zona_2"] = [z2[i].varValue for i in df.index]
df["zona_3"] = [z3[i].varValue for i in df.index]
df["zona_optima"] = df.apply(lambda row: 2 if row["zona_2"] == 1 else (3 if row["zona_3"] == 1 else 0), axis=1)

print(df)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/diegovertiz/Documents/AACuarto Semestre/Optimización Determinista/venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/jb/15trx49d5b13y0zq5089zsp40000gn/T/b85d66a3c6cd4fb5b77d3a769da974cd-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/jb/15trx49d5b13y0zq5089zsp40000gn/T/b85d66a3c6cd4fb5b77d3a769da974cd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 593 COLUMNS
At line 2064 RHS
At line 2653 BOUNDS
At line 2948 ENDATA
Problem MODEL has 588 rows, 294 columns and 588 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 248.222 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -2

In [7]:
import pandas as pd
import numpy as np

# Carga el DataFrame
df = pd.read_csv("flujo_efectivo_tienda_dia.csv")

# Guardamos una copia base para reutilizar
df_base = df.copy()

# Lista para guardar resultados
resultados = []

# Rango de tiempo de reposición a evaluar
for t in np.arange(1, 10.5, 0.5):
    df = df_base.copy()
    
    # Calcular nuevo ROP con el t actual
    rop_por_tienda = df.groupby("tienda")["salida_prestamos"].mean().reset_index()
    rop_por_tienda["ROP_efectivo"] = rop_por_tienda["salida_prestamos"] * t

    # Unir con df
    df = df.merge(rop_por_tienda[["tienda", "ROP_efectivo"]], on="tienda", how="left")
    
    # Recalcular zona
    df["Max"] = 250000
    df["Min"] = df["ROP_efectivo"] * 0.6
    df["PO"] = (df["ROP_efectivo"] + df["Max"]) / 2
    
    condiciones = [
        df["flujo_neto"] < df["Min"],
        (df["flujo_neto"] >= df["Min"]) & (df["flujo_neto"] < df["ROP_efectivo"]),
        (df["flujo_neto"] >= df["ROP_efectivo"]) & (df["flujo_neto"] < df["PO"]),
        (df["flujo_neto"] >= df["PO"]) & (df["flujo_neto"] <= df["Max"]),
        df["flujo_neto"] > df["Max"]
    ]
    zonas = [0, 1, 2, 3, 4]
    df["Zona"] = np.select(condiciones, zonas)

    # Contar cuántas están en zona 2 o 3
    score = ((df["Zona"] == 2) | (df["Zona"] == 3)).sum()
    
    resultados.append((t, score))

# Convertir a DataFrame
resultados_df = pd.DataFrame(resultados, columns=["t", "tiendas_en_zona_optima"])

# Mostrar mejor t
mejor_fila = resultados_df.loc[resultados_df["tiendas_en_zona_optima"].idxmax()]
print(f"🔍 Mejor t: {mejor_fila['t']} con {mejor_fila['tiendas_en_zona_optima']} tiendas en zonas 2 y 3")

# Graficar resultado
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(resultados_df["t"], resultados_df["tiendas_en_zona_optima"], marker='o')
plt.axvline(mejor_fila['t'], color='red', linestyle='--', label=f"Mejor t = {mejor_fila['t']}")
plt.xlabel("Tiempo de Reposición (t)")
plt.ylabel("Tiendas en Zona 2 o 3")
plt.title("Optimización del Tiempo de Reposición para Maximizar Tiendas Óptimas")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

KeyError: 'ROP_efectivo'